# Principal component analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from poliparties.datasets import ches2019
from poliparties import analysis, plot

from sklearn import decomposition, impute, preprocessing
from sklearn.experimental import enable_iterative_imputer

In [ ]:
#
# Load dataset
#
(X_train, features) = ches2019.prepare(
    ches2019.cleanup(
        # ches2019.update(),
        ches2019.load(),
        nan_floor_row=0.9,
        nan_floor_col=0.75
    )
)

# Scaling to reasonable units
scaler = analysis.IntervalScaler([
    v for (k, v) in ches2019.feature_scales.items() if k in features
])
X_train = scaler.transform(X_train)

# Impute missing values
imputer = impute.IterativeImputer().fit(X_train)
X_train = imputer.transform(X_train)

In [ ]:
fig = plt.figure(figsize=(12, 10))
ax = plot.plot_training_data(
    fig.gca(),
    X_train,
    features,
    aspect="auto",
    cmap="cubehelix"
)

In [ ]:
#
# Fit decpomposition
#

# Principal component analysis
decomposer = decomposition.PCA(2).fit(X_train)

Y = decomposer.transform(X_train)
Y_2d = Y[:, :2]

U = decomposer.components_ 
V = scaler.inverse_transform(U)

(fig, ax) = plt.subplots(figsize=(6, 8))
ax = plot.plot_components(ax, V[:2], features)
_ = ax.set_title(
    ax.get_title() + "\n Explained variance ratio {}".format(
        decomposer.explained_variance_ratio_[:2].sum()
    )
)

In [ ]:
#
# Estimating the 2d probability density
# 

# Fit KDE and sample
kde = analysis.fit_kde(Y_2d)
num_samples = 10
Y_samples = kde.sample(num_samples)
sample_colors = plot.create_colors(
    num_samples, plt.cm.gist_rainbow
)  # for plotting

# Evaluate probability density in grid for plotting
(x, y, density, xlim, ylim) = analysis.score_density_grid(kde=kde, Y=Y_2d, num=100)


#
# Plot
#

(fig, axs) = plt.subplots(1, 2, figsize=(8, 4))

ax = axs[0]
ax.contourf(x, y, density, levels=10, cmap=plt.cm.cubehelix, alpha=0.9)
ax.set_title("Estimated probability density")

ax = axs[1]
ax.scatter(*Y_2d.T, alpha=0.5, c="k")
_ = ax.set_xlim(xlim)
_ = ax.set_ylim(ylim)
ax.set_title("Projected observations")

(fig, ax) = plt.subplots(figsize=(8, 8))
ax.scatter(*Y_samples.T, c=sample_colors, s=100)
ax.contour(x, y, density, levels=10, cmap="cubehelix")
ax.grid(True)
_ = ax.set_xlim(xlim)
_ = ax.set_ylim(ylim)
ax.set_title("Random samples from estimated distribution")

In [ ]:
#
# Mapping back to original dimensions
#

X_samples = scaler.inverse_transform(
    decomposer.inverse_transform(Y_samples)
)

(fig, ax) = plt.subplots(figsize=(8, 8))
ax = plot.plot_components(
    ax, 
    X_samples, 
    features, 
    label="Sample",
    colors=sample_colors
)